# DenseNet - KHOTAA Diabetic Foot Ulcer Classification

## 1. Imports & Configuration

In [ ]:
# TO BE IMPLEMENTED
# Import necessary libraries
# from torchvision.models import densenet121, DenseNet121_Weights

## 2. Load Dataset

In [ ]:
# TO BE IMPLEMENTED

## 3. Model Definition

In [ ]:
# TO BE IMPLEMENTED
# Define DenseNet model

## 4. Training

In [ ]:
# TO BE IMPLEMENTED

## 5. Evaluation & Plots

In [ ]:
# TO BE IMPLEMENTED